In [126]:
# make the target list, now based on ratio between rolling lightcurve SD and (updated) stacked/mean fit depth

# import packages
import batman
import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
import lightkurve as lk
from scipy.optimize import curve_fit
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
from lmfit import Model
from lmfit import Parameter
import pandas as pd
import csv
import os
import sys
import datetime
from astroquery.simbad import Simbad
import random
import argparse

parser = argparse.ArgumentParser(description="Get mean rolling lightcurve SD.")
parser.add_argument("-start", "--start", type=int, help="Index of planet to start on.")
parser.add_argument("-stop", "--stop", type=int, help="Index of planet to stop on.")
parser.add_argument("-outfile", "--outfile", type=str, help="output file.")
args = parser.parse_args()
start = args.start
stop = args.stop
outfile = args.outfile


# NASA exoplanet archive table
NASA_exoplanet_table = pd.read_csv('NASA_exoplanet_archive.csv', skiprows = 292)

# Expected, mean fit depth, and mean flux err table
depth_err_tab = pd.read_csv('KOI_depth_err_ratios_updated.csv') 

# remove trends in baseline flux data
def linear_func(x, a, b):
  return a*x + b

def stitch(lc_collection): 

    tot_time = np.zeros(0) # to store concatenated arrays
    tot_flux = np.zeros(0)
    tot_flux_err = np.zeros(0)
    tot_qual = np.zeros(0)
    for i in range(len(lc_collection)):
        lc = lc_collection[i]
        flux = lc.flux.value
        time = lc.time.value
        flux_err = lc.flux_err.value
        qual = lc.quality
        rel_flux_err = flux_err/flux
        nan_mask = np.invert(np.isnan(flux))
        
        # Fit and remove linear trend
        popt, pcov = curve_fit(linear_func, time[nan_mask], flux[nan_mask])
        linear_trend = linear_func(time, *popt) # evaluate over the whole interval
        norm = flux / linear_trend
        
        tot_time = np.hstack([tot_time, time])
        tot_flux = np.hstack([tot_flux, norm])
        tot_flux_err = np.hstack([tot_flux_err, rel_flux_err])
        tot_qual = np.hstack([tot_qual, qual])
        
    return tot_time, tot_flux, tot_flux_err, tot_qual

  # define fit function
def inj_func(time, t0, period, RpRs, a, inc, ecc, w, u1, u2):#, low_bound, up_bound):

    # set up a transit object
    params = batman.TransitParams()
    params.t0 = t0                 # time of inferior conjunction
    params.per = period            # orbital period [days]
    params.rp = RpRs               # planet radius (in units of stellar radii) [Rp/R*]
    params.a = a                   # semi-major axis (in units of stellar radii)
    params.inc = inc               # orbital inclination (in degrees)
    params.ecc = ecc               # eccentricity
    params.w = w                   # longitude of periastron (in degrees) -- documentation says argument of periapse [deg]
    params.u = [u1, u2]            # limb darkening coefficients [u1, u2]
    params.limb_dark = "quadratic" # limb darkening model

    # initialze model + calculate lightcurve
    #interval = (low_bound < time)*(time < up_bound) # interval to fit on
    t = time#[interval]                              # times to calculate at
    m = batman.TransitModel(params, t)              # initializes model
    model_flux = m.light_curve(params)              # calculates light curve

    return model_flux


def func(x, t0, period, RpRs, a, inc, ecc, w, u1, u2, low_bound, up_bound):

    # set up a transit object
    params = batman.TransitParams()
    params.t0 = t0                 # time of inferior conjunction
    params.per = period            # orbital period [days]
    params.rp = RpRs               # planet radius (in units of stellar radii) [Rp/R*]
    params.a = a                   # semi-major axis (in units of stellar radii)
    params.inc = inc               # orbital inclination (in degrees)
    params.ecc = ecc               # eccentricity
    params.w = w                   # longitude of periastron (in degrees) -- documentation says argument of periapse [deg]
    params.u = [u1, u2]            # limb darkening coefficients [u1, u2]
    params.limb_dark = "quadratic" # limb darkening model

    # initialze model + calculate lightcurve
    interval = (low_bound < time)*(time < up_bound) # interval to fit on
    t = time[interval]                              # times to calculate at
    m = batman.TransitModel(params, t)              # initializes model
    model_flux = m.light_curve(params)              # calculates light curve

    #print(t
    return model_flux


mean_rolling_SD = np.zeros(len(kep_names)) * np.nan
for i in kep_names[start:stop]:
    planet_name =  kep_names[i]

    if planet_name.startswith('kepler-') or planet_name.startswith('Kepler-'):
        planet_name = planet_name.replace('k','K')
        star_name = planet_name[:-1].replace(' ', '')
        obs_type = planet_name.split('-')[0]

    else:
        star_name = input('Enter star name: ')
        obs_type = input('Enter whether this is a \'kepler\' or \'TESS\' observation: ')


    print(planet_name + ': ')

    try: 
        planet = NasaExoplanetArchive.query_object(planet_name)
        nasa_duration = np.nanmedian(planet['pl_trandur']).value / 24 # Planet transit duration [Days]
    except: # NASA exoplanet archive cannot currenlty be queried via astroquery (known bug but fix still pending)

        query_name = planet_name
        if query_name == 'Kepler-1 b':  query_name = 'TrES-2 b'
        if query_name == 'Kepler-458 b': query_name = 'KIC 9663113 b'
        if query_name == 'Kepler-324 d': query_name = 'KOI-1831 d'    
        if query_name == 'Kepler-1703 b':  query_name = 'KOI-3503 b'
        if query_name == 'Kepler-968 d':  query_name = 'KOI-1833 d'
        if query_name == 'Kepler-460 b':  query_name = 'KIC 5437945 b'
        if query_name == 'Kepler-86 b':  query_name = 'PH2 b'
        if query_name == 'Kepler-1703 c':  query_name = 'KOI-3503 c' 

        print('querying with name: ' + query_name)

        planet =  NASA_exoplanet_table[NASA_exoplanet_table['pl_name'] == query_name]

        if planet.empty:
            print('tying alternate names.')
            result_table = Simbad.query_objectids(planet_name)
            alt_names = result_table.to_pandas()
            alt_names = list(alt_names.iloc[:,0].str.decode('utf-8')) # gets rid of weird formatting
            for new_name in alt_names:
                #new_name = new_name[0].split(' ')[-1]
                if new_name[-1].islower():
                    new_name = new_name[:-1] + ' ' + new_name[-1]
               #   try:
                planet =  NASA_exoplanet_table[NASA_exoplanet_table['pl_name'] == new_name]
                if not planet.empty:
                    break 
             #   except: continue
        nasa_duration = np.nanmedian(planet['pl_trandur']) / 24 # Planet transit duration [Days]

    # search for and download all lightcurves
    search_result = lk.search_lightcurve(star_name, author=obs_type, cadence='long')
    lc_collection = search_result.download_all()
    lc = lc_collection.stitch() # stitch their way becuase ned to do bls #.remove_outliers() # it turns out removing outliers can remove transits as well! As can flattening with a poor window choice. 
    crowdsap = lc.meta['CROWDSAP'] # A measure of contamination from nearby stars
    time, flux, flux_err, quality = stitch(lc_collection)
    dt = np.median(np.diff(time))  # [Days] Usual timestep size (should match cadence, except for example missing data)
    flux_SD = np.std(flux)         # Standard deviation in flux
    mean_flux_err = np.nanmean(flux_err)  # Mean error in flux 

    # get cadence information
    cadence_type = lc.meta['OBSMODE'] 
    if obs_type == 'Kepler' or obs_type == 'kepler':
      if cadence_type == 'long cadence':
          cadence = 1764 # sec
      if cadence_type == 'short cadence':
          cadence = 58.85 # sec  


    # mask out transits of other planets in the system
    try:
        system_results = NasaExoplanetArchive.query_object(star_name)
        planets = system_results['pl_letter'].pformat()[2:]
        unique_planets = set([planet for planet in list(set(planets)) if (not planet.startswith('Length')) and (not planet.endswith('.'))])
        #unique_planets.remove('Length = 24 rows')
        system_periods = list(system_results['pl_orbper'])
        system_t0 = list(system_results['pl_tranmid']) 
        system_dur = list(system_results['pl_trandur']) 
        no_mask_flux = np.copy(flux)
        no_mask_time = np.copy(time)
        no_mask_quality = np.copy(quality)
        mask = np.array([False]) * len(flux)
        bls = lc.to_periodogram(method='bls', period=np.linspace(1,50), frequency_factor=500) # need this to use lightkurve's masking function
        for planet in unique_planets: 
            if planet_name[-1] != planet[-1]:
                print('masking other transits.')
                unique_period = np.nanmedian(system_periods[planets.index(planet)]).value
                unique_t0 = np.nanmedian(system_t0[planets.index(planet)]).value - 2454833
                if np.isnan(np.nanmedian(system_dur[planets.index(planet)])):
                    continue
                unique_duration = np.nanmedian(system_dur[planets.index(planet)]).value / 24
                planet_mask = bls.get_transit_mask(period = unique_period, transit_time = unique_t0, duration = unique_duration)
                mask = np.logical_or(mask,planet_mask)  
    except:
        system_results = NASA_exoplanet_table[NASA_exoplanet_table['hostname'] == star_name]
        planets = list(system_results['pl_letter'])
        unique_planets = set([planet for planet in list(set(planets)) if (not planet.startswith('Length')) and (not planet.endswith('.'))])
        #unique_planets.remove('Length = 24 rows')
        system_periods = list(system_results['pl_orbper'])
        system_t0 = list(system_results['pl_tranmid']) 
        system_dur = list(system_results['pl_trandur']) 
        no_mask_flux = np.copy(flux)
        no_mask_flux_err = np.copy(flux_err)
        no_mask_time = np.copy(time)
        no_mask_quality = np.copy(quality)
        mask = np.zeros(len(flux)).astype(bool)
        bls = lc.to_periodogram(method='bls', period=np.linspace(1,50), frequency_factor=500) # need this to use lightkurve's masking function
        for planet in unique_planets: 
          #  if planet_name[-1] != planet[-1]: # for injections, mask out ALL transits
            unique_period = np.nanmedian(system_periods[planets.index(planet)])
            unique_t0 = np.nanmedian(system_t0[planets.index(planet)]) - 2454833
            print('masking planet: ' + planet)
            if np.isnan(np.nanmedian(system_dur[planets.index(planet)])):
                continue
            unique_duration = np.nanmedian(system_dur[planets.index(planet)]) / 24
            #planet_mask = bls.get_transit_mask(period = unique_period, transit_time = unique_t0, duration = unique_duration)
            # in the above line, bls.get_transit_mask seems to remove outliers, thus often resulting in a different length mask than the lighcurve itself
            planet_mask = lc.create_transit_mask(period=float(unique_period), transit_time=float(unique_t0), duration=float(unique_duration))
            mask = np.logical_or(mask,planet_mask)  
            print('masked planet: ' + planet)

    flux = flux[np.invert(mask)]
    flux_err = flux_err[np.invert(mask)]
    time = time[np.invert(mask)]
    quality_no_mask = np.copy(quality)
    quality = quality[np.invert(mask)]


    # assesss quality flags and remove bad data
    # try a more lenient quality cut
    no_quality_cut = (quality >= 0)
    quality_cut_lenient = (quality != 16) * (quality != 128) * (quality != 2048) 
    quality_cut_medium = (quality != 1) * (quality != 4) * (quality != 16) * (quality != 32) * (quality != 128) * (quality != 256) * (quality != 2048) * (quality != 32768)
    quality_cut_aggressive = (quality == 0) # this is the original, default cut
    good_data_flag = quality_cut_aggressive * (flux_err > 0) * (np.isfinite(time)) * (np.isfinite(flux)) * (np.isfinite(flux_err)) 
    bad_time = time[np.invert(good_data_flag)]
    bad_flux = flux[np.invert(good_data_flag)]     
    bad_flux_err = flux_err[np.invert(good_data_flag)]     
    time = time[good_data_flag]
    flux = flux[good_data_flag]
    flux_err = flux_err[good_data_flag]

    df = pd.DataFrame(flux)
    mean_rolling_SD[i] = np.nanmean(df.rolling(10).std())
    

df2 = pd.DataFrame(mean_rolling_SD)    
df2.to_csv(outfile, index = False)  